In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Input, LeakyReLU, BatchNormalization, LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.layers.core import Dense, Activation, Dropout

In [17]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [18]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')

data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [19]:
n_steps = 5
x_data, y_data = split_sequence(data, n_steps)

In [20]:
x_data = x_data.reshape((x_data.shape[0], -1))
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data = x_data.reshape((x_data.shape[0], -1, 1))
x_data.shape

(421, 25, 1)

In [21]:
final_predict_data = x_data[-1:,:].copy()

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=0.9, random_state=66, shuffle=True
)

In [25]:
inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
x1 = LSTM(128, activation='relu')(inp)
x1 = Dense(256)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.4)(x1)
x1 = Dense(128)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(64)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
out = Dense(1)(x1)

model = Model(inputs=inp, outputs=out)
# 훈련
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=100, batch_size=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
378/378 [==============================] - 4s 12ms/step - loss: 19869193831.1958 - acc: 0.0000e+00
Epoch 2/100
378/378 [==============================] - 4s 10ms/step - loss: 2158893576.8042 - acc: 0.0000e+00
Epoch 3/100
378/378 [==============================] - 4s 10ms/step - loss: 14384588929.0159 - acc: 0.0000e+00
Epoch 4/100
378/378 [==============================] - 4s 10ms/step - loss: 1956350301.9683 - acc: 0.0000e+00
Epoch 5/100
378/378 [==============================] - 4s 10ms/step - loss: 29045679901.1217 - acc: 0.0000e+00
Epoch 6/100
378/378 [==============================] - 4s 10ms/step - loss: 1824716300.1905 - acc: 0.0000e+00
Epoch 7/100
378/378 [==============================] - 4s 10ms/step - loss: 1744646420.3175 - acc: 0.0000e+00
Epoch 8/100
378/378 [==============================] - 4s 10ms/step - loss: 1583747781.4180 - acc: 0.0000e+00
Epoch 9/100
378/378 [==============================] - 4s 10ms/step -

378/378 [==============================] - 4s 10ms/step - loss: 16569248.5351 - acc: 0.0000e+00
Epoch 74/100
378/378 [==============================] - 4s 10ms/step - loss: 17464157.2016 - acc: 0.0000e+00
Epoch 75/100
378/378 [==============================] - 4s 10ms/step - loss: 17599020.7101 - acc: 0.0000e+00
Epoch 76/100
378/378 [==============================] - 4s 10ms/step - loss: 18452175.9807 - acc: 0.0000e+00
Epoch 77/100
378/378 [==============================] - 4s 10ms/step - loss: 18625857.1973 - acc: 0.0000e+00
Epoch 78/100
378/378 [==============================] - 4s 10ms/step - loss: 16658914.9747 - acc: 0.0000e+00
Epoch 79/100
378/378 [==============================] - 4s 10ms/step - loss: 17783308.6182 - acc: 0.0000e+00
Epoch 80/100
378/378 [==============================] - 4s 10ms/step - loss: 17488634.4982 - acc: 0.0000e+00
Epoch 81/100
378/378 [==============================] - 4s 10ms/step - loss: 17301465.6734 - acc: 0.0000e+00
Epoch 82/100
378/378 [==========

In [26]:
y_predict = model.predict(x_test)

In [27]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 2127.3026287327707


In [28]:
print(model.predict(final_predict_data))

[[49455.402]]
